# Notebook Overview

This notebook focuses on handling and visualizing Statsbomb event data, specifically for public data related to soccer matches. The main steps include:

1.  **Configuration**: Setting up the environment by installing necessary libraries.
2.  **Data Loading and Filtering**: Retrieving competition and match data from the Statsbomb API and filtering it based on specific criteria (e.g., gender, age, year, season).
3.  **Retrieving and Processing Event Data**: Obtaining detailed event data for the selected matches, handling it in chunks, and performing initial data wrangling.
4.  **Analyzing Event Data**: Analyzing different types of events (Duels, 50/50s, Dispossessed, Foul Committed, Foul Won) and calculating relevant metrics like sequence length and passes in sequence.
5.  **Combining Event Data**: Merging the analyzed data from different event types into a single DataFrame.
6.  **Saving the Combined Data**: Saving the final processed data to Google Drive for further use.

This notebook aims to provide a foundation for exploring and visualizing soccer event data from Statsbomb.

# Introduction

Handle Statsbomb event data and explore visualization with public data.

# Configuration

In [ ]:
!pip install statsbombpy

In [ ]:
import pandas as pd
from statsbombpy import sb



In [ ]:
a=0
b=1

The statsbombpy automatically interact with repository. For more details you can access to github.


https://github.com/statsbomb/open-data

Data Loading and Filtering

This section focuses on loading the necessary data from the Statsbomb API and filtering it to include only relevant competitions and matches for analysis.

First, we retrieve a list of all available competitions from Statsbomb. We then filter this list to include only male, non-international, non-youth competitions from the year 2000 onwards.

Next, we retrieve all matches for the filtered competitions and concatenate them into a single DataFrame. Finally, we filter these matches further to focus on the 2015/2016 season and exclude Champions League matches, as per the project's requirements.

In [ ]:
df_competitions = sb.competitions()
df_competitions.head()

/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-09-28T20:46:38.893391,2025-07-06T04:26:07.636270,2025-07-06T04:26:07.636270,2024-09-28T20:46:38.893391
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-09-28T01:57:35.846538,None,None,2024-09-28T01:57:35.846538
3,16,4,Europe,Champions League,male,False,False,2018/2019,2025-05-08T15:10:50.835274,2021-06-13T16:17:31.694,None,2025-05-08T15:10:50.835274
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [ ]:

df_competitions['season_year'] = df_competitions['season_name'].str[-4:].astype(int)
df_competitions_filtered = df_competitions[
    (df_competitions['season_year'] > 2000) &
    (df_competitions['competition_international'] == False) &
    (df_competitions['competition_youth'] == False) &
    (df_competitions['competition_gender'] == 'male')
]
df_competitions_filtered

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available,season_year
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-09-28T20:46:38.893391,2025-07-06T04:26:07.636270,2025-07-06T04:26:07.636270,2024-09-28T20:46:38.893391,2024
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381,2016
3,16,4,Europe,Champions League,male,False,False,2018/2019,2025-05-08T15:10:50.835274,2021-06-13T16:17:31.694,None,2025-05-08T15:10:50.835274,2019
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882,2018
5,16,2,Europe,Champions League,male,False,False,2016/2017,2024-02-13T02:37:32.205154,2021-06-13T16:17:31.694,None,2024-02-13T02:37:32.205154,2017
6,16,27,Europe,Champions League,male,False,False,2015/2016,2024-06-12T07:45:38.786894,2021-06-13T16:17:31.694,None,2024-06-12T07:45:38.786894,2016
7,16,26,Europe,Champions League,male,False,False,2014/2015,2024-02-12T12:49:54.914228,2021-06-13T16:17:31.694,None,2024-02-12T12:49:54.914228,2015
8,16,25,Europe,Champions League,male,False,False,2013/2014,2024-02-12T12:48:48.479157,2021-06-13T16:17:31.694,None,2024-02-12T12:48:48.479157,2014
9,16,24,Europe,Champions League,male,False,False,2012/2013,2024-02-12T12:47:34.340413,2021-06-13T16:17:31.694,None,2024-02-12T12:47:34.340413,2013
10,16,23,Europe,Champions League,male,False,False,2011/2012,2024-02-13T02:36:35.698340,2021-06-13T16:17:31.694,None,2024-02-13T02:36:35.698340,2012


In [ ]:


all_matches = []
for index, row in df_competitions_filtered.iterrows():
    competition_id = row['competition_id']
    season_id = row['season_id']
    try:
        df_matches = sb.matches(competition_id=competition_id, season_id=season_id)
        all_matches.append(df_matches)
    except Exception as e:
        print(f"Could not retrieve matches for competition_id: {competition_id}, season_id: {season_id}. Error: {e}")

if all_matches:
    df_all_matches = pd.concat(all_matches, ignore_index=True)
    print(df_all_matches.head())
else:
    print("No matches data retrieved.")


df_all_matches

/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: 

   match_id  match_date      kick_off              competition     season  \
0   3895302  2024-04-14  17:30:00.000  Germany - 1. Bundesliga  2023/2024   
1   3895292  2024-04-06  15:30:00.000  Germany - 1. Bundesliga  2023/2024   
2   3895333  2024-05-05  18:30:00.000  Germany - 1. Bundesliga  2023/2024   
3   3895340  2024-05-12  20:30:00.000  Germany - 1. Bundesliga  2023/2024   
4   3895348  2024-05-18  16:30:00.000  Germany - 1. Bundesliga  2023/2024   

             home_team         away_team  home_score  away_score match_status  \
0     Bayer Leverkusen     Werder Bremen           5           0    available   
1         Union Berlin  Bayer Leverkusen           0           1    available   
2  Eintracht Frankfurt  Bayer Leverkusen           1           5    available   
3               Bochum  Bayer Leverkusen           0           5    available   
4     Bayer Leverkusen          Augsburg           2           1    available   

   ...            last_updated_360 match_week comp

/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3895302,2024-04-14,17:30:00.000,Germany - 1. Bundesliga,2023/2024,Bayer Leverkusen,Werder Bremen,5,0,available,...,2024-05-10T17:03:59.613154,29,Regular Season,BayArena,Harm Osmers,Xabier Alonso Olano,Ole Werner,1.1.0,2,2
1,3895292,2024-04-06,15:30:00.000,Germany - 1. Bundesliga,2023/2024,Union Berlin,Bayer Leverkusen,0,1,available,...,2024-05-12T21:08:37.897296,28,Regular Season,Stadion An der Alten Försterei,Benjamin Brand,Nenad Bjelica,Xabier Alonso Olano,1.1.0,2,2
2,3895333,2024-05-05,18:30:00.000,Germany - 1. Bundesliga,2023/2024,Eintracht Frankfurt,Bayer Leverkusen,1,5,available,...,2024-05-14T16:32:13.483516,32,Regular Season,Deutsche Bank Park,Christian Dingert,Dino Toppmöller,Xabier Alonso Olano,1.1.0,2,2
3,3895340,2024-05-12,20:30:00.000,Germany - 1. Bundesliga,2023/2024,Bochum,Bayer Leverkusen,0,5,available,...,2024-05-14T16:46:08.459843,33,Regular Season,Vonovia Ruhrstadion,Benjamin Brand,Heiko Butscher,Xabier Alonso Olano,1.1.0,2,2
4,3895348,2024-05-18,16:30:00.000,Germany - 1. Bundesliga,2023/2024,Bayer Leverkusen,Augsburg,2,1,available,...,2024-05-20T10:33:09.140760,34,Regular Season,BayArena,Matthias Jöllenbeck,Xabier Alonso Olano,Jess Christian Thorup,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,3878544,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Palermo,Genoa,1,0,available,...,None,1,Regular Season,Stadio Renzo Barbera,Massimiliano Irrati,Giuseppe Iachini,Gian Piero Gasperini,1.1.0,2,2
2571,3878543,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Inter Milan,Atalanta,1,0,available,...,None,1,Regular Season,Stadio Giuseppe Meazza,Gianpaolo Calvarese\t,Roberto Mancini,Edoardo Reja,1.1.0,2,2
2572,3878542,2015-08-23,20:45:00.000,Italy - Serie A,2015/2016,Fiorentina,AC Milan,2,0,available,...,None,1,Regular Season,Stadio Artemio Franchi \t,Paolo Valeri,Paulo Manuel Carvalho de Sousa,Siniša Mihajlović,1.1.0,2,2
2573,3878541,2015-08-22,18:00:00.000,Italy - Serie A,2015/2016,Hellas Verona,AS Roma,1,1,available,...,None,1,Regular Season,Stadio Marc''Antonio Bentegodi,Marco Guida,"Rafael Márquez Álvarez, Andrea Mandorlini",Rudi Garcia,1.1.0,2,2


In [ ]:
df_filtered_matches = df_all_matches[
    (df_all_matches['season']=='2015/2016') &
    (df_all_matches['competition'] != 'Europe - Champions League')
].copy()
print(df_filtered_matches.head())

    match_id  match_date      kick_off              competition     season  \
34   3890561  2016-05-14  15:30:00.000  Germany - 1. Bundesliga  2015/2016   
35   3890505  2016-04-02  15:30:00.000  Germany - 1. Bundesliga  2015/2016   
36   3890511  2016-04-08  20:30:00.000  Germany - 1. Bundesliga  2015/2016   
37   3890515  2016-04-09  15:30:00.000  Germany - 1. Bundesliga  2015/2016   
38   3890411  2015-12-20  16:30:00.000  Germany - 1. Bundesliga  2015/2016   

        home_team            away_team  home_score  away_score match_status  \
34     Hoffenheim           Schalke 04           1           4    available   
35  Bayern Munich  Eintracht Frankfurt           1           0    available   
36  Hertha Berlin          Hannover 96           2           2    available   
37   Hamburger SV         Darmstadt 98           1           2    available   
38  Hertha Berlin         FSV Mainz 05           2           0    available   

    ... last_updated_360 match_week competition_stage   

In [ ]:

df_selected_cols = df_filtered_matches[['competition', 'match_id','season']]

In [ ]:

df_2015_2016 = df_filtered_matches[df_filtered_matches['season'] == '2015/2016'].copy()

# Group by competition and list match_ids
competition_match_ids = df_2015_2016.groupby('competition')['match_id'].apply(list)

competition_match_ids

,match_id
competition,
England - Premier League,"[3754058, 3754245, 3754136, 3754037, 3754039, ..."
France - Ligue 1,"[3901199, 3901178, 3901260, 3829470, 3901259, ..."
Germany - 1. Bundesliga,"[3890561, 3890505, 3890511, 3890515, 3890411, ..."
Italy - Serie A,"[3879863, 3879773, 3879847, 3879862, 3879817, ..."
Spain - La Liga,"[3825848, 3825895, 3825894, 3825855, 3825908, ..."


In [ ]:
df_wwc = df_filtered_matches.copy()

In [ ]:
matchids=df_wwc['match_id'].unique()
print(matchids)

[3890561 3890505 3890511 ... 3878542 3878541 3878540]


In [ ]:

# Group match ids into chunks
chunk_size = 250
match_id_chunks = [matchids[i:i + chunk_size] for i in range(0, len(matchids), chunk_size)]

# Print the names of each group (by index)
for i, chunk in enumerate(match_id_chunks):
  print(f"Group {i+1}")
  # If you want to see the match ids in each group, uncomment the line below
  print(chunk)

Group 1
[3890561 3890505 3890511 3890515 3890411 3890397 3890401 3890396 3890384
 3890385 3890282 3890287 3890269 3890268 3890402 3890564 3890563 3890562
 3890560 3890559 3890558 3890557 3890556 3890555 3890554 3890553 3890552
 3890551 3890550 3890549 3890548 3890547 3890546 3890545 3890544 3890543
 3890542 3890541 3890540 3890539 3890538 3890537 3890536 3890535 3890534
 3890533 3890532 3890531 3890530 3890529 3890528 3890527 3890526 3890525
 3890524 3890523 3890522 3890521 3890520 3890519 3890518 3890517 3890516
 3890514 3890513 3890512 3890510 3890509 3890508 3890507 3890506 3890504
 3890503 3890502 3890501 3890500 3890499 3890498 3890497 3890496 3890495
 3890494 3890493 3890492 3890491 3890490 3890489 3890488 3890487 3890486
 3890485 3890484 3890483 3890482 3890481 3890480 3890479 3890478 3890477
 3890476 3890475 3890474 3890473 3890472 3890471 3890470 3890469 3890468
 3890467 3890466 3890465 3890464 3890463 3890462 3890461 3890460 3890459
 3890458 3890457 3890456 3890455 3890454 38

## Retrieving and Processing Event Data

This section focuses on retrieving the detailed event data for the selected matches and performing initial processing to prepare it for analysis.

Since retrieving event data for all matches at once can be resource-intensive, we group the match IDs into smaller chunks and retrieve the events for each chunk individually.

After retrieving the event data, we concatenate it into a single DataFrame and perform some initial data wrangling steps, such as extracting location coordinates and creating a unique identifier for each possession.

In [ ]:

# Select the first chunk of match IDs
match_ids_chunk_1 = match_id_chunks[a]

# Retrieve events for the matches in the first chunk
all_events_chunk_1 = []
for match_id in match_ids_chunk_1:
  try:
    df_events = sb.events(match_id=match_id)
    all_events_chunk_1.append(df_events)
  except Exception as e:
    print(f"Could not retrieve events for match_id: {match_id}. Error: {e}")

# Concatenate the events dataframes from the first chunk
if all_events_chunk_1:
  df_all_events_chunk_1 = pd.concat(all_events_chunk_1, ignore_index=True)
  print(df_all_events_chunk_1.head())
else:
  print("No events data retrieved for the first chunk of matches.")

# Display the dataframe
df_all_events_chunk_1


/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsbombpy/api_client.py:21: 

  ball_receipt_outcome ball_recovery_recovery_failure block_deflection  \
0                  NaN                            NaN              NaN   
1                  NaN                            NaN              NaN   
2                  NaN                            NaN              NaN   
3                  NaN                            NaN              NaN   
4                  NaN                            NaN              NaN   

  carry_end_location clearance_aerial_won clearance_body_part clearance_head  \
0                NaN                  NaN                 NaN            NaN   
1                NaN                  NaN                 NaN            NaN   
2                NaN                  NaN                 NaN            NaN   
3                NaN                  NaN                 NaN            NaN   
4                NaN                  NaN                 NaN            NaN   

  clearance_left_foot clearance_right_foot counterpress  ... block_save_bl

,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,block_save_block,goalkeeper_punched_out,shot_redirect,goalkeeper_shot_saved_off_target,shot_saved_off_target,goalkeeper_shot_saved_to_post,shot_saved_to_post,goalkeeper_lost_out,goalkeeper_success_in_play,shot_follows_dribble
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_final_matches = df_all_events_chunk_1


In [ ]:

df_final_matches = pd.merge(df_final_matches, df_wwc[['match_id', 'home_team']], on='match_id', how='left')


In [ ]:

df_final_matches['home_team_flag'] = (df_final_matches['possession_team'] == df_final_matches['home_team']).astype(int)


In [ ]:
df_final_matches[['x','y']] = df_final_matches['location'].apply(pd.Series)

In [ ]:

df_final_matches['new_id'] = df_final_matches['match_id'].astype(str) + '_' + df_final_matches['possession'].astype(str)
df_first_event_of_possession = df_final_matches.loc[df_final_matches.groupby('new_id')['index'].idxmin()]

In [ ]:
print(df_final_matches['type'].unique())

['Starting XI' 'Half Start' 'Pass' 'Ball Receipt*' 'Carry' 'Pressure'
 'Block' 'Ball Recovery' 'Miscontrol' 'Interception' 'Foul Committed'
 'Foul Won' 'Shot' 'Goal Keeper' 'Duel' 'Dribble' 'Dribbled Past'
 'Clearance' 'Dispossessed' 'Tactical Shift' 'Injury Stoppage'
 'Player Off' 'Player On' 'Half End' 'Substitution' 'Shield'
 'Own Goal Against' 'Own Goal For' '50/50' 'Referee Ball-Drop'
 'Bad Behaviour' 'Offside' 'Error']


In [ ]:

new_dfs = {}
for event_type in df_final_matches['type'].unique():
  df_name = f"df_{event_type.replace(' ', '_').lower()}"
  new_dfs[df_name] = df_final_matches[df_final_matches['type'] == event_type].copy()

print("List of new dataframes:")
for df_name in new_dfs.keys():
    # Indent this line to be inside the loop and use print() to display the name
    print(df_name)

# You can access a specific dataframe like this:
df_carry = new_dfs['df_carry']
print(df_carry.head())

List of new dataframes:
df_starting_xi
df_half_start
df_pass
df_ball_receipt*
df_carry
df_pressure
df_block
df_ball_recovery
df_miscontrol
df_interception
df_foul_committed
df_foul_won
df_shot
df_goal_keeper
df_duel
df_dribble
df_dribbled_past
df_clearance
df_dispossessed
df_tactical_shift
df_injury_stoppage
df_player_off
df_player_on
df_half_end
df_substitution
df_shield
df_own_goal_against
df_own_goal_for
df_50/50
df_referee_ball-drop
df_bad_behaviour
df_offside
df_error
     ball_receipt_outcome ball_recovery_recovery_failure block_deflection  \
2165                  NaN                            NaN              NaN   
2166                  NaN                            NaN              NaN   
2167                  NaN                            NaN              NaN   
2168                  NaN                            NaN              NaN   
2169                  NaN                            NaN              NaN   

     carry_end_location clearance_aerial_won clearance_body

**High Level**

In [ ]:

match_analysis = df_final_matches.groupby('match_id').agg(
    duration=('duration', 'sum'),
    xg=('shot_statsbomb_xg', 'sum'),
    home_team_duration=('duration', lambda x: x[df_final_matches.loc[x.index, 'home_team_flag'] == 1].sum()),
    max_posession=('possession', 'max')
).reset_index()

# Calculate the shot_home_proportion after aggregation
match_analysis['home_team_duration'] = match_analysis['home_team_duration'] / match_analysis['duration']

match_analysis

,match_id,duration,xg,home_team_duration,max_posession
0,3890268,3105.198468,1.704714,0.493212,172
1,3890269,2586.192081,3.186915,0.586176,213
2,3890282,3051.304171,4.665310,0.499207,177
3,3890287,3287.595991,3.982188,0.824313,143
4,3890319,3672.900467,1.960017,0.694971,167
...,...,...,...,...,...
245,3890560,3317.339631,2.348650,0.326420,181
246,3890561,3559.617941,2.774578,0.446149,171
247,3890562,3492.866025,2.532932,0.839255,144
248,3890563,2748.193509,6.738388,0.431430,205


**df_duel**

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card',
    'team',
    'x',
    'y',
    'counterpress',
    'index',
    'play_pattern'
]
# Check if the columns exist in the database
df_duel = new_dfs['df_duel']

existing_columns = [col for col in selected_columns if col in df_duel.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_duel = df_duel[existing_columns]
df_final_matches_selected_duel

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern
3749,3c73cb65-1e7c-4f75-ae9d-9d58c17096da,3890561,"[40.1, 28.0]",1,8,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,40.1,28.0,NaN,277,From Goal Kick
3750,f86304b1-9544-4e98-858c-745112e13220,3890561,"[40.5, 49.7]",1,10,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,40.5,49.7,True,324,Regular Play
3751,b99ea45b-bb32-4737-acdd-f0956f0c6404,3890561,"[31.0, 25.5]",1,14,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,31.0,25.5,True,376,Regular Play
3752,aa044964-a84f-4dca-a086-bace1265ddf7,3890561,"[9.1, 9.0]",1,14,Hoffenheim,True,Hoffenheim,1,NaN,Schalke 04,9.1,9.0,NaN,390,Regular Play
3753,df845598-2464-439a-b672-1dc3c43bcbc1,3890561,"[47.0, 53.7]",1,17,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,47.0,53.7,NaN,435,From Goal Kick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879290,db753ae2-a51f-42db-b717-a48eb8a9df06,3890319,"[70.8, 63.1]",2,160,Hannover 96,True,Wolfsburg,0,NaN,Wolfsburg,70.8,63.1,NaN,3742,From Keeper
879291,d85955d8-fbbd-4fb9-9af4-cddac7c052ee,3890319,"[81.7, 62.2]",2,162,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,81.7,62.2,NaN,3799,From Goal Kick
879292,07d01043-f697-4bf2-a787-2e64d500fbe0,3890319,"[91.2, 70.6]",2,162,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,91.2,70.6,NaN,3806,From Goal Kick
879293,ab741c4c-c513-4342-8a58-0e06934bae14,3890319,"[78.1, 9.7]",2,164,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,78.1,9.7,NaN,3840,From Goal Kick


In [ ]:
# Join df_final_matches_selected_foul_committed and df_final_matches on 'match_id' and 'possession'
df_merged = pd.merge(df_final_matches_selected_duel[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_foul_committed
df_final_matches_selected_duel = pd.merge(df_final_matches_selected_duel,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

df_final_matches_selected_duel

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern,min_index
0,3c73cb65-1e7c-4f75-ae9d-9d58c17096da,3890561,"[40.1, 28.0]",1,8,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,40.1,28.0,NaN,277,From Goal Kick,265
1,f86304b1-9544-4e98-858c-745112e13220,3890561,"[40.5, 49.7]",1,10,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,40.5,49.7,True,324,Regular Play,300
2,b99ea45b-bb32-4737-acdd-f0956f0c6404,3890561,"[31.0, 25.5]",1,14,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,31.0,25.5,True,376,Regular Play,376
3,aa044964-a84f-4dca-a086-bace1265ddf7,3890561,"[9.1, 9.0]",1,14,Hoffenheim,True,Hoffenheim,1,NaN,Schalke 04,9.1,9.0,NaN,390,Regular Play,376
4,df845598-2464-439a-b672-1dc3c43bcbc1,3890561,"[47.0, 53.7]",1,17,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,47.0,53.7,NaN,435,From Goal Kick,414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21168,db753ae2-a51f-42db-b717-a48eb8a9df06,3890319,"[70.8, 63.1]",2,160,Hannover 96,True,Wolfsburg,0,NaN,Wolfsburg,70.8,63.1,NaN,3742,From Keeper,3711
21169,d85955d8-fbbd-4fb9-9af4-cddac7c052ee,3890319,"[81.7, 62.2]",2,162,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,81.7,62.2,NaN,3799,From Goal Kick,3797
21170,07d01043-f697-4bf2-a787-2e64d500fbe0,3890319,"[91.2, 70.6]",2,162,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,91.2,70.6,NaN,3806,From Goal Kick,3797
21171,ab741c4c-c513-4342-8a58-0e06934bae14,3890319,"[78.1, 9.7]",2,164,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,78.1,9.7,NaN,3840,From Goal Kick,3838


In [ ]:
df_final_matches_selected_duel = df_final_matches_selected_duel.sort_values(by=['match_id', 'possession', 'index'])
df_final_matches_selected_duel['sequence length'] = df_final_matches_selected_duel['index'] - df_final_matches_selected_duel['min_index']

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_foul_committed
for index, row in df_final_matches_selected_duel.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_foul_committed
df_final_matches_selected_duel['passes_in_sequence'] = pass_counts

print(df_final_matches_selected_duel[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:")

df_final_matches_selected_duel = pd.merge(df_final_matches_selected_duel,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_final_matches_selected_duel.head())
df_final_matches_selected_duel['field'] = "Duel"

     match_id  index  min_index  passes_in_sequence
933   3890268     96         47                  16
934   3890268    164        103                  21
935   3890268    165        103                  21
936   3890268    187        103                  28
937   3890268    191        103                  30

First few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:
                                     id  match_id      location  period  \
0  a7fa7dea-ca81-4c64-a4ea-10aa8ce3e195   3890268  [53.0, 41.1]       1   
1  d098b93c-c2f7-42c9-a01a-9160a176ab03   3890268  [63.7, 27.8]       1   
2  bbc7293c-ea45-46e9-b676-971e2c2a8ff5   3890268  [62.2, 51.8]       1   
3  f2b02261-48d8-48ec-bc77-afdb55480215   3890268  [56.7, 21.5]       1   
4  1e13799d-a7c8-47d0-bad7-ce9e5206b8f4   3890268  [40.4, 11.1]       1   

   possession possession_team under_pressure      home_team  home_team_flag  \
0           3   Werder Bremen           True  Hertha Berlin       

**50/50**

In [ ]:
selected_columns = [

    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card',
    'team',
    'x',
    'y',
    'counterpress',
    'index',
    'play_pattern'
]
# Check if the columns exist in the database
df_50 = new_dfs['df_50/50']

existing_columns = [col for col in selected_columns if col in df_duel.columns]

# Create a new dataframe with only the existing selected columns
df_50 = df_50[existing_columns]
df_50

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern
7447,ca2faf73-a331-4f22-a016-3c09d777b4d8,3890505,"[66.7, 56.5]",1,26,Bayern Munich,True,Bayern Munich,1,NaN,Eintracht Frankfurt,66.7,56.5,NaN,475,Regular Play
10876,7481e322-cad4-4422-9df6-599e7fd14861,3890511,"[64.9, 9.4]",1,21,Hertha Berlin,True,Hertha Berlin,1,NaN,Hannover 96,64.9,9.4,NaN,497,Regular Play
10877,68e09fba-8321-40ba-89f9-3ee81aaee642,3890511,"[55.2, 70.7]",1,21,Hertha Berlin,True,Hertha Berlin,1,NaN,Hertha Berlin,55.2,70.7,NaN,498,Regular Play
10878,8320e3a9-c85d-4dda-a909-f31b0ae1d658,3890511,"[53.1, 4.0]",2,181,Hertha Berlin,True,Hertha Berlin,1,NaN,Hertha Berlin,53.1,4.0,NaN,3372,Regular Play
10879,7b7718c8-379e-4495-9b44-86284818fa2a,3890511,"[67.0, 76.1]",2,181,Hertha Berlin,True,Hertha Berlin,1,NaN,Hannover 96,67.0,76.1,NaN,3373,Regular Play
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879700,64f1f539-2f09-4181-8d99-d1be4ad18a3f,3890319,"[97.7, 40.6]",2,84,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,97.7,40.6,NaN,2114,From Kick Off
879701,47066f6d-c17d-4c4b-8485-ee3ffed93ddf,3890319,"[95.0, 59.6]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,95.0,59.6,NaN,2730,Regular Play
879702,b190bb3c-c97d-4584-9170-87bb5a777238,3890319,"[25.1, 20.5]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Hannover 96,25.1,20.5,NaN,2731,Regular Play
879703,2342e97f-9495-469f-ad77-5923755eff3f,3890319,"[35.2, 26.8]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Hannover 96,35.2,26.8,NaN,2732,Regular Play


In [ ]:
df_merged = pd.merge(df_50[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_foul_committed
df_50 = pd.merge(df_50,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

df_50

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern,min_index
0,ca2faf73-a331-4f22-a016-3c09d777b4d8,3890505,"[66.7, 56.5]",1,26,Bayern Munich,True,Bayern Munich,1,NaN,Eintracht Frankfurt,66.7,56.5,NaN,475,Regular Play,450
1,7481e322-cad4-4422-9df6-599e7fd14861,3890511,"[64.9, 9.4]",1,21,Hertha Berlin,True,Hertha Berlin,1,NaN,Hannover 96,64.9,9.4,NaN,497,Regular Play,482
2,68e09fba-8321-40ba-89f9-3ee81aaee642,3890511,"[55.2, 70.7]",1,21,Hertha Berlin,True,Hertha Berlin,1,NaN,Hertha Berlin,55.2,70.7,NaN,498,Regular Play,482
3,8320e3a9-c85d-4dda-a909-f31b0ae1d658,3890511,"[53.1, 4.0]",2,181,Hertha Berlin,True,Hertha Berlin,1,NaN,Hertha Berlin,53.1,4.0,NaN,3372,Regular Play,3361
4,7b7718c8-379e-4495-9b44-86284818fa2a,3890511,"[67.0, 76.1]",2,181,Hertha Berlin,True,Hertha Berlin,1,NaN,Hannover 96,67.0,76.1,NaN,3373,Regular Play,3361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,64f1f539-2f09-4181-8d99-d1be4ad18a3f,3890319,"[97.7, 40.6]",2,84,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,97.7,40.6,NaN,2114,From Kick Off,1969
1231,47066f6d-c17d-4c4b-8485-ee3ffed93ddf,3890319,"[95.0, 59.6]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,95.0,59.6,NaN,2730,Regular Play,2707
1232,b190bb3c-c97d-4584-9170-87bb5a777238,3890319,"[25.1, 20.5]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Hannover 96,25.1,20.5,NaN,2731,Regular Play,2707
1233,2342e97f-9495-469f-ad77-5923755eff3f,3890319,"[35.2, 26.8]",2,106,Wolfsburg,True,Wolfsburg,1,NaN,Hannover 96,35.2,26.8,NaN,2732,Regular Play,2707


In [ ]:
df_50 = df_50.sort_values(by=['match_id', 'possession', 'index'])
df_50['sequence length'] = df_50['index'] - df_50['min_index']

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_foul_committed
for index, row in df_50.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_foul_committed
df_50['passes_in_sequence'] = pass_counts

print(df_50[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:")

df_50 = pd.merge(df_50,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_50.head())
df_50['field'] = "50/50"

    match_id  index  min_index  passes_in_sequence
45   3890268    769        763                   1
46   3890268    770        763                   1
47   3890268   2025       2008                   4
48   3890268   2026       2008                   4
49   3890268   2215       2199                   6

First few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:
                                     id  match_id       location  period  \
0  7f0451a5-6844-45db-82f7-bbffd7b04942   3890268   [13.6, 50.5]       1   
1  5aba3af9-c42b-4228-8a4b-147b6e62cc88   3890268  [106.5, 29.6]       1   
2  00c1df75-ffba-490b-95aa-b7332105ab09   3890268   [85.7, 38.4]       2   
3  63d510e1-6e0c-48d6-93c5-70b468275345   3890268   [34.4, 41.7]       2   
4  ac76b486-f617-426f-ad73-2f0119166469   3890268   [70.0, 50.6]       2   

   possession possession_team under_pressure      home_team  home_team_flag  \
0          44   Werder Bremen           True  Hertha Berlin       

**df_dispossessed** - reviewed

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card',
    'team',
    'x',
    'y',
    'counterpress',
    'index',
    'play_pattern'
]
# Check if the columns exist in the database
df_dispossessed = new_dfs['df_dispossessed']

existing_columns = [col for col in selected_columns if col in df_dispossessed.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_dispossessed = df_dispossessed[existing_columns]
df_final_matches_selected_dispossessed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern
3893,b87722f9-5324-498e-8525-4cb4040653f0,3890561,"[69.9, 23.1]",1,23,Hoffenheim,True,Hoffenheim,1,NaN,Schalke 04,69.9,23.1,NaN,564,From Free Kick
3894,03dc0915-4ceb-4ab6-be47-ea9e4c37c2ca,3890561,"[62.6, 49.9]",1,33,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,62.6,49.9,NaN,719,Regular Play
3895,562ee383-2ba3-479d-8a25-1f64e1165519,3890561,"[115.7, 13.4]",1,36,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,115.7,13.4,NaN,832,From Free Kick
3896,87251742-2fa2-4cc5-baf3-270402d56562,3890561,"[4.8, 74.5]",1,37,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,4.8,74.5,NaN,837,Regular Play
3897,35d5f1e6-2918-4f39-9aec-d79177c779b6,3890561,"[108.6, 46.1]",1,39,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,108.6,46.1,NaN,887,Regular Play
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879599,a1977316-ba12-43cf-a835-7d487e3d3f38,3890319,"[83.1, 76.4]",2,132,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,83.1,76.4,NaN,3182,Regular Play
879600,e2ee46a9-70fd-43eb-b9b7-968cd5d1c22a,3890319,"[89.0, 74.8]",2,136,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,89.0,74.8,NaN,3328,Regular Play
879601,bd37b89b-ad0a-4917-8a8c-b0f1c3047ad6,3890319,"[76.5, 5.3]",2,148,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,76.5,5.3,NaN,3537,From Goal Kick
879602,dc97cc22-ae79-4428-80e4-26ec3f8398cd,3890319,"[50.2, 5.6]",2,159,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,50.2,5.6,NaN,3705,From Throw In


In [ ]:
# Join df_final_matches_selected_foul_committed and df_final_matches on 'match_id' and 'possession'
df_merged = pd.merge(df_final_matches_selected_dispossessed[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_foul_committed
df_final_matches_selected_dispossessed = pd.merge(df_final_matches_selected_dispossessed,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

df_final_matches_selected_dispossessed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern,min_index
0,b87722f9-5324-498e-8525-4cb4040653f0,3890561,"[69.9, 23.1]",1,23,Hoffenheim,True,Hoffenheim,1,NaN,Schalke 04,69.9,23.1,NaN,564,From Free Kick,560
1,03dc0915-4ceb-4ab6-be47-ea9e4c37c2ca,3890561,"[62.6, 49.9]",1,33,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,62.6,49.9,NaN,719,Regular Play,689
2,562ee383-2ba3-479d-8a25-1f64e1165519,3890561,"[115.7, 13.4]",1,36,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,115.7,13.4,NaN,832,From Free Kick,810
3,87251742-2fa2-4cc5-baf3-270402d56562,3890561,"[4.8, 74.5]",1,37,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,4.8,74.5,NaN,837,Regular Play,833
4,35d5f1e6-2918-4f39-9aec-d79177c779b6,3890561,"[108.6, 46.1]",1,39,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,108.6,46.1,NaN,887,Regular Play,859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,a1977316-ba12-43cf-a835-7d487e3d3f38,3890319,"[83.1, 76.4]",2,132,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,83.1,76.4,NaN,3182,Regular Play,3161
6298,e2ee46a9-70fd-43eb-b9b7-968cd5d1c22a,3890319,"[89.0, 74.8]",2,136,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,89.0,74.8,NaN,3328,Regular Play,3292
6299,bd37b89b-ad0a-4917-8a8c-b0f1c3047ad6,3890319,"[76.5, 5.3]",2,148,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,76.5,5.3,NaN,3537,From Goal Kick,3530
6300,dc97cc22-ae79-4428-80e4-26ec3f8398cd,3890319,"[50.2, 5.6]",2,159,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,50.2,5.6,NaN,3705,From Throw In,3701


In [ ]:
df_final_matches_selected_dispossessed = df_final_matches_selected_dispossessed.sort_values(by=['match_id', 'possession', 'index'])
df_final_matches_selected_dispossessed['sequence length'] = df_final_matches_selected_dispossessed['index'] - df_final_matches_selected_dispossessed['min_index']

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_foul_committed
for index, row in df_final_matches_selected_dispossessed.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_foul_committed
df_final_matches_selected_dispossessed['passes_in_sequence'] = pass_counts

print(df_final_matches_selected_dispossessed[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:")

df_final_matches_selected_dispossessed = pd.merge(df_final_matches_selected_dispossessed,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_final_matches_selected_dispossessed.head())
df_final_matches_selected_dispossessed['field'] = "Dispossessed"

     match_id  index  min_index  passes_in_sequence
279   3890268    163        103                  21
280   3890268    203        195                   1
281   3890268    243        238                   1
282   3890268    399        377                   7
283   3890268    540        512                   8

First few rows of df_final_matches_selected_dispossessed with 'passes_in_sequence' column:
                                     id  match_id       location  period  \
0  a9d9561d-7d27-48d0-bed8-5cf0d6a06c83   3890268   [56.4, 52.3]       1   
1  d6da819e-021b-4686-9c90-fa9b3debb0b5   3890268  [112.5, 56.9]       1   
2  6edf750a-11f3-4768-be71-82d435b1f7fc   3890268   [79.5, 78.3]       1   
3  f17cb533-7746-4f1a-ae50-d169b42b0582   3890268   [69.2, 73.0]       1   
4  055af85d-d813-478d-8996-0fdaa1646adc   3890268   [53.9, 31.4]       1   

   possession possession_team under_pressure      home_team  home_team_flag  \
0           4   Hertha Berlin           True  Hertha Berlin 

In [ ]:
df_final_matches_selected_dispossessed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,...,y,counterpress,index,play_pattern,min_index,sequence length,passes_in_sequence,competition,season,field
0,a9d9561d-7d27-48d0-bed8-5cf0d6a06c83,3890268,"[56.4, 52.3]",1,4,Hertha Berlin,True,Hertha Berlin,1,NaN,...,52.3,NaN,163,From Free Kick,103,60,21,Germany - 1. Bundesliga,2015/2016,Dispossessed
1,d6da819e-021b-4686-9c90-fa9b3debb0b5,3890268,"[112.5, 56.9]",1,5,Hertha Berlin,True,Hertha Berlin,1,NaN,...,56.9,NaN,203,From Throw In,195,8,1,Germany - 1. Bundesliga,2015/2016,Dispossessed
2,6edf750a-11f3-4768-be71-82d435b1f7fc,3890268,"[79.5, 78.3]",1,9,Werder Bremen,True,Hertha Berlin,0,NaN,...,78.3,NaN,243,From Throw In,238,5,1,Germany - 1. Bundesliga,2015/2016,Dispossessed
3,f17cb533-7746-4f1a-ae50-d169b42b0582,3890268,"[69.2, 73.0]",1,18,Hertha Berlin,True,Hertha Berlin,1,NaN,...,73.0,NaN,399,From Throw In,377,22,7,Germany - 1. Bundesliga,2015/2016,Dispossessed
4,055af85d-d813-478d-8996-0fdaa1646adc,3890268,"[53.9, 31.4]",1,25,Hertha Berlin,True,Hertha Berlin,1,NaN,...,31.4,NaN,540,From Free Kick,512,28,8,Germany - 1. Bundesliga,2015/2016,Dispossessed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ea13bf08-91fb-4bd8-8ef1-c6e620ef6271,3890564,"[77.1, 19.9]",2,166,Hamburger SV,True,Augsburg,0,NaN,...,19.9,NaN,2882,From Free Kick,2877,5,1,Germany - 1. Bundesliga,2015/2016,Dispossessed
6298,ad43fcd3-dfec-4309-bc81-078b783de09a,3890564,"[105.3, 62.7]",2,168,Hamburger SV,True,Augsburg,0,NaN,...,62.7,NaN,2980,Regular Play,2899,81,22,Germany - 1. Bundesliga,2015/2016,Dispossessed
6299,f926d744-d517-4cfb-922c-014d22262b9d,3890564,"[19.9, 7.5]",2,169,Hamburger SV,True,Augsburg,0,NaN,...,7.5,NaN,2992,From Throw In,2983,9,1,Germany - 1. Bundesliga,2015/2016,Dispossessed
6300,3dbf0855-8d17-4a16-872d-6e48e8d6d3e2,3890564,"[115.1, 12.5]",2,169,Hamburger SV,True,Augsburg,0,NaN,...,12.5,NaN,3009,From Throw In,2983,26,4,Germany - 1. Bundesliga,2015/2016,Dispossessed


**Foul Committed**

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card',
    'team',
    'x',
    'y',
    'counterpress',
    'index',
    'play_pattern',


]
# Check if the columns exist in the database
df_foul_committed = new_dfs['df_foul_committed']

existing_columns = [col for col in selected_columns if col in df_foul_committed.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_foul_committed = df_foul_committed[existing_columns]
df_final_matches_selected_foul_committed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern
3632,f1dd3418-5813-4df3-b319-97e1f821a4bb,3890561,"[52.0, 55.2]",1,7,Hoffenheim,NaN,Hoffenheim,1,NaN,Schalke 04,52.0,55.2,NaN,200,Regular Play
3633,e0b0ab13-9eed-4a18-aadb-c73c194c6f6a,3890561,"[91.3, 76.3]",1,19,Schalke 04,NaN,Hoffenheim,0,NaN,Hoffenheim,91.3,76.3,True,477,Regular Play
3634,bd08408b-0d22-40b0-a6ce-38bff67eacc5,3890561,"[105.4, 70.7]",1,20,Schalke 04,NaN,Hoffenheim,0,NaN,Schalke 04,105.4,70.7,NaN,498,From Free Kick
3635,4487ead1-4686-4618-8781-30dbabbf6a0a,3890561,"[114.8, 21.4]",1,22,Schalke 04,NaN,Hoffenheim,0,NaN,Schalke 04,114.8,21.4,True,558,From Goal Kick
3636,0f56d472-b60a-4ced-b3c7-eb261eed470b,3890561,"[87.1, 30.8]",1,25,Hoffenheim,NaN,Hoffenheim,1,NaN,Schalke 04,87.1,30.8,True,601,Regular Play
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879621,bec69705-458f-4fad-b0f2-ce1b1ecf60fa,3890319,"[58.6, 12.1]",2,129,Wolfsburg,NaN,Wolfsburg,1,NaN,Hannover 96,58.6,12.1,NaN,3116,From Throw In
879622,0f57793c-4d5e-4ed0-ae23-4df87ea36a01,3890319,"[40.3, 37.6]",2,146,Wolfsburg,NaN,Wolfsburg,1,Yellow Card,Hannover 96,40.3,37.6,NaN,3506,From Throw In
879623,72f82593-ba2c-46a5-8c12-de9b95c7fb92,3890319,"[55.1, 73.4]",2,155,Hannover 96,NaN,Wolfsburg,0,NaN,Wolfsburg,55.1,73.4,NaN,3640,From Throw In
879624,29bac64e-733f-4865-9fa4-c3a5bf62c5db,3890319,"[38.7, 25.4]",2,164,Hannover 96,NaN,Wolfsburg,0,NaN,Wolfsburg,38.7,25.4,NaN,3856,From Goal Kick


In [ ]:
# Join df_final_matches_selected_foul_committed and df_final_matches on 'match_id' and 'possession'
df_merged = pd.merge(df_final_matches_selected_foul_committed[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_foul_committed
df_final_matches_selected_foul_committed = pd.merge(df_final_matches_selected_foul_committed,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

df_final_matches_selected_foul_committed

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern,min_index
0,f1dd3418-5813-4df3-b319-97e1f821a4bb,3890561,"[52.0, 55.2]",1,7,Hoffenheim,NaN,Hoffenheim,1,NaN,Schalke 04,52.0,55.2,NaN,200,Regular Play,192
1,e0b0ab13-9eed-4a18-aadb-c73c194c6f6a,3890561,"[91.3, 76.3]",1,19,Schalke 04,NaN,Hoffenheim,0,NaN,Hoffenheim,91.3,76.3,True,477,Regular Play,475
2,bd08408b-0d22-40b0-a6ce-38bff67eacc5,3890561,"[105.4, 70.7]",1,20,Schalke 04,NaN,Hoffenheim,0,NaN,Schalke 04,105.4,70.7,NaN,498,From Free Kick,479
3,4487ead1-4686-4618-8781-30dbabbf6a0a,3890561,"[114.8, 21.4]",1,22,Schalke 04,NaN,Hoffenheim,0,NaN,Schalke 04,114.8,21.4,True,558,From Goal Kick,519
4,0f56d472-b60a-4ced-b3c7-eb261eed470b,3890561,"[87.1, 30.8]",1,25,Hoffenheim,NaN,Hoffenheim,1,NaN,Schalke 04,87.1,30.8,True,601,Regular Play,599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7996,bec69705-458f-4fad-b0f2-ce1b1ecf60fa,3890319,"[58.6, 12.1]",2,129,Wolfsburg,NaN,Wolfsburg,1,NaN,Hannover 96,58.6,12.1,NaN,3116,From Throw In,3109
7997,0f57793c-4d5e-4ed0-ae23-4df87ea36a01,3890319,"[40.3, 37.6]",2,146,Wolfsburg,NaN,Wolfsburg,1,Yellow Card,Hannover 96,40.3,37.6,NaN,3506,From Throw In,3495
7998,72f82593-ba2c-46a5-8c12-de9b95c7fb92,3890319,"[55.1, 73.4]",2,155,Hannover 96,NaN,Wolfsburg,0,NaN,Wolfsburg,55.1,73.4,NaN,3640,From Throw In,3629
7999,29bac64e-733f-4865-9fa4-c3a5bf62c5db,3890319,"[38.7, 25.4]",2,164,Hannover 96,NaN,Wolfsburg,0,NaN,Wolfsburg,38.7,25.4,NaN,3856,From Goal Kick,3838


In [ ]:
df_final_matches_selected_foul_committed = df_final_matches_selected_foul_committed.sort_values(by=['match_id', 'possession', 'index'])
df_final_matches_selected_foul_committed['sequence length'] = df_final_matches_selected_foul_committed['index'] - df_final_matches_selected_foul_committed['min_index']

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_foul_committed
for index, row in df_final_matches_selected_foul_committed.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_foul_committed
df_final_matches_selected_foul_committed['passes_in_sequence'] = pass_counts

print(df_final_matches_selected_foul_committed[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_foul_committed with 'passes_in_sequence' column:")

df_final_matches_selected_foul_committed = pd.merge(df_final_matches_selected_foul_committed,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_final_matches_selected_foul_committed.head())
df_final_matches_selected_foul_committed['field'] = "Foul_Committed"

     match_id  index  min_index  passes_in_sequence
399   3890268    454        451                   0
400   3890268    510        507                   1
401   3890268    640        638                   0
402   3890268    648        642                   2
403   3890268    761        731                   8

First few rows of df_final_matches_selected_foul_committed with 'passes_in_sequence' column:
                                     id  match_id       location  period  \
0  b6765dd6-5036-4b36-a2dc-4883429248de   3890268  [109.5, 45.9]       1   
1  bfa5fb2b-742c-404d-b936-0b6a47c3ed18   3890268   [77.4, 35.0]       1   
2  7c95cf04-993b-441a-8051-de57061cb9e9   3890268   [92.0, 65.0]       1   
3  178a8f01-1244-482a-b1c2-603e7c348268   3890268   [33.4, 17.5]       1   
4  71a82806-5ef9-4881-b843-33a708640507   3890268   [50.9, 16.3]       1   

   possession possession_team under_pressure      home_team  home_team_flag  \
0          21   Hertha Berlin            NaN  Hertha Berli

**df_foul_won** - on pause

In [ ]:
selected_columns = [
    'id',
    'match_id',
    'location',
    'period',
    'possession',
    'possession_team',
    'under_pressure',
    'home_team',
    'home_team_flag',
    'foul_committed_card',
    'team',
    'x',
    'y',
    'counterpress',
    'index',
    'play_pattern'
]
# Check if the columns exist in the database
df_foul_won = new_dfs['df_foul_won']

existing_columns = [col for col in selected_columns if col in df_foul_won.columns]

# Create a new dataframe with only the existing selected columns
df_final_matches_selected_foul_won = df_foul_won[existing_columns]
df_final_matches_selected_foul_won

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern
3662,7c25c22a-275c-40a1-b2c6-399c3f5663d5,3890561,"[68.1, 24.9]",1,7,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,68.1,24.9,NaN,201,Regular Play
3663,40613828-de11-4af8-babd-4a4874656e05,3890561,"[28.8, 3.8]",1,19,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,28.8,3.8,NaN,478,Regular Play
3664,dad95065-2241-4a71-bed6-4d3ea2b5d65a,3890561,"[14.7, 9.4]",1,20,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,14.7,9.4,NaN,499,From Free Kick
3665,7234bf2e-fbaa-4cad-b414-b8fd14707d8b,3890561,"[5.3, 58.7]",1,22,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,5.3,58.7,NaN,559,From Goal Kick
3666,961ce5da-3b1d-41cf-bbe9-46be8cae6540,3890561,"[33.0, 49.3]",1,25,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,33.0,49.3,NaN,602,Regular Play
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879642,e5e81120-1ed3-4e67-a1d6-94df869c9624,3890319,"[88.7, 61.0]",2,125,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,88.7,61.0,NaN,3086,From Throw In
879643,c2f7b724-b0a6-4c90-863b-42943009b291,3890319,"[79.8, 42.5]",2,146,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,79.8,42.5,NaN,3507,From Throw In
879644,df99191b-6302-4263-8ac8-8323eba9752d,3890319,"[65.0, 6.7]",2,155,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,65.0,6.7,NaN,3641,From Throw In
879645,7be2ad88-bafa-4b5f-bd55-09d5aa11240f,3890319,"[81.4, 54.7]",2,164,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,81.4,54.7,NaN,3857,From Goal Kick


In [ ]:
# Join df_final_matches_selected_foul_committed and df_final_matches on 'match_id' and 'possession'
df_merged = pd.merge(df_final_matches_selected_foul_won[['match_id', 'possession']],
                     df_final_matches[['match_id', 'possession', 'index']],
                     on=['match_id', 'possession'],
                     how='inner')

# Find the minimum index for each match_id and possession combination
df_min_index = df_merged.groupby(['match_id', 'possession'])['index'].min().reset_index()
df_min_index.rename(columns={'index': 'min_index'}, inplace=True)

# Join the minimum index back onto df_final_matches_selected_foul_committed
df_final_matches_selected_foul_won = pd.merge(df_final_matches_selected_foul_won,
                                          df_min_index,
                                          on=['match_id', 'possession'],
                                          how='left')

df_final_matches_selected_foul_won

,id,match_id,location,period,possession,possession_team,under_pressure,home_team,home_team_flag,foul_committed_card,team,x,y,counterpress,index,play_pattern,min_index
0,7c25c22a-275c-40a1-b2c6-399c3f5663d5,3890561,"[68.1, 24.9]",1,7,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,68.1,24.9,NaN,201,Regular Play,192
1,40613828-de11-4af8-babd-4a4874656e05,3890561,"[28.8, 3.8]",1,19,Schalke 04,True,Hoffenheim,0,NaN,Schalke 04,28.8,3.8,NaN,478,Regular Play,475
2,dad95065-2241-4a71-bed6-4d3ea2b5d65a,3890561,"[14.7, 9.4]",1,20,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,14.7,9.4,NaN,499,From Free Kick,479
3,7234bf2e-fbaa-4cad-b414-b8fd14707d8b,3890561,"[5.3, 58.7]",1,22,Schalke 04,True,Hoffenheim,0,NaN,Hoffenheim,5.3,58.7,NaN,559,From Goal Kick,519
4,961ce5da-3b1d-41cf-bbe9-46be8cae6540,3890561,"[33.0, 49.3]",1,25,Hoffenheim,True,Hoffenheim,1,NaN,Hoffenheim,33.0,49.3,NaN,602,Regular Play,599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7680,e5e81120-1ed3-4e67-a1d6-94df869c9624,3890319,"[88.7, 61.0]",2,125,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,88.7,61.0,NaN,3086,From Throw In,3072
7681,c2f7b724-b0a6-4c90-863b-42943009b291,3890319,"[79.8, 42.5]",2,146,Wolfsburg,True,Wolfsburg,1,NaN,Wolfsburg,79.8,42.5,NaN,3507,From Throw In,3495
7682,df99191b-6302-4263-8ac8-8323eba9752d,3890319,"[65.0, 6.7]",2,155,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,65.0,6.7,NaN,3641,From Throw In,3629
7683,7be2ad88-bafa-4b5f-bd55-09d5aa11240f,3890319,"[81.4, 54.7]",2,164,Hannover 96,True,Wolfsburg,0,NaN,Hannover 96,81.4,54.7,NaN,3857,From Goal Kick,3838


In [ ]:
df_final_matches_selected_foul_won = df_final_matches_selected_foul_won.sort_values(by=['match_id', 'possession', 'index'])
df_final_matches_selected_foul_won['sequence length'] = df_final_matches_selected_foul_won['index'] - df_final_matches_selected_foul_won['min_index']

# Initialize a list to store the counts for each shot
pass_counts = []

# Iterate through each row in df_final_matches_selected_foul_committed
for index, row in df_final_matches_selected_foul_won.iterrows():
    match_id = row['match_id']
    shot_index = row['index']
    min_index = row['min_index']

    # Filter df_final_matches for the current match_id and the relevant index range
    relevant_events = df_final_matches[
        (df_final_matches['match_id'] == match_id) &
        (df_final_matches['index'] >= min_index) &
        (df_final_matches['index'] <= shot_index)
    ]

    # Count the number of rows where 'type' is 'Pass' in the filtered events
    count_passes = relevant_events[relevant_events['type'] == 'Pass'].shape[0]

    # Append the count to the list
    pass_counts.append(count_passes)

# Add the pass counts as a new column to df_final_matches_selected_foul_committed
df_final_matches_selected_foul_won['passes_in_sequence'] = pass_counts

print(df_final_matches_selected_foul_won[['match_id', 'index', 'min_index', 'passes_in_sequence']].head())
print("\nFirst few rows of df_final_matches_selected_foul_committed with 'passes_in_sequence' column:")

df_final_matches_selected_foul_won = pd.merge(df_final_matches_selected_foul_won,
                                          df_selected_cols[['match_id', 'competition', 'season']],
                                          on='match_id',
                                          how='left')
print(df_final_matches_selected_foul_won.head())
df_final_matches_selected_foul_won['field'] = "Foul_Won"

     match_id  index  min_index  passes_in_sequence
384   3890268    455        451                   0
385   3890268    511        507                   1
386   3890268    641        638                   0
387   3890268    649        642                   2
388   3890268    762        731                   8

First few rows of df_final_matches_selected_foul_committed with 'passes_in_sequence' column:
                                     id  match_id      location  period  \
0  b982b850-ab91-44b7-aa17-f71de69a4f47   3890268  [10.6, 34.2]       1   
1  85b75f93-8c35-4651-a42c-c43756e81917   3890268  [42.7, 45.1]       1   
2  b2fa81a1-e163-4012-a65f-9d4ef69ce5d9   3890268  [28.1, 15.1]       1   
3  fb01cbb2-35ca-4bac-ad85-423a4f77f877   3890268  [86.7, 62.6]       1   
4  fdd69cd0-fbd2-4cc6-83fa-ed892e8b2d58   3890268  [69.2, 63.8]       1   

   possession possession_team under_pressure      home_team  home_team_flag  \
0          21   Hertha Berlin           True  Hertha Berlin     

## Combining Event Data for Analysis

This section combines the dataframes created for each specific event type (Duels, 50/50s, Dispossessed, Foul Committed, and Foul Won) into a single DataFrame for further analysis related to referee decisions and game events.

In [ ]:
# List of dataframes to join
referee_analysis = [
df_final_matches_selected_foul_won,
df_final_matches_selected_foul_committed,
df_final_matches_selected_dispossessed,
df_final_matches_selected_duel,
df_50
]

df_referee_analysis = pd.concat(referee_analysis, ignore_index=True)

## Saving the Combined Analysis Data

This section saves the combined DataFrame containing the analysis of various event types (`df_referee_analysis`) to a CSV file in your Google Drive for future use or further analysis.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# Specify the path where you want to save the CSV file in your Google Drive

file_path = f'/content/drive/My Drive/referee_analysis{b}.csv'
# Save the DataFrame to a CSV file
df_referee_analysis.to_csv(file_path, index=False)
print(f"DataFrame saved successfully to {file_path}")

MessageError: Error: credential propagation was unsuccessful